In [1]:
import arviz as az
import numpy as np
import polars as ps
import pymc as pm
from pytensor import shared
from sklearn.model_selection import train_test_split

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Load Data

In [3]:
type_map: list = [ps.Int32, ps.Int32, ps.Int32, ps.Int32, ps.Int32, ps.Int32, ps.Int32, 
                  ps.Int32, ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Float32, 
                  ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Float32, 
                  ps.Float32, ps.Int32, ps.Float32, ps.Float32, ps.Float32]

In [4]:
dataset: ps.DataFrame = ps.read_csv('full_dataset_train.csv', dtypes=type_map, encoding='utf8', n_threads=8, use_pyarrow=True)

In [5]:
dataset = dataset[:1000]

In [6]:
dataset.head()

SiteId,PublishYear,PublishMonth,PublishDay,PublishHour,AQI,Pollutant,Status,SO2,SO2_AVG,CO,CO_8hr,O3,O3_8hr,PM10,PM10_AVG,NO,NO2,NOx,WindDirec,WindSpeed,PM2.5,PM2.5_AVG
i32,i32,i32,i32,i32,i32,i32,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i32,f32,f32,f32
48,2022,6,1,0,27,0,0,0.9,1.0,0.3,0.1,1.7,14.3,19.0,17.0,3.9,16.5,20.4,85,0.9,7.0,5.0
58,2022,6,1,1,40,0,0,2.1,3.0,0.1,0.1,0.4,3.3,21.0,16.0,22.4,24.6,47.0,237,0.4,13.0,8.0
71,2022,6,1,1,37,0,0,2.1,1.0,0.3,0.2,0.0,7.2,22.0,18.0,6.0,22.200001,28.200001,253,0.5,8.0,7.0
15,2022,6,1,2,23,1,0,0.7,1.0,0.4,0.5,0.7,3.4,16.0,13.0,3.7,14.0,17.799999,70,0.3,9.0,7.0
8,2022,6,1,4,23,1,0,0.6,0.0,0.3,0.5,4.2,3.6,14.0,12.0,1.5,10.3,11.8,161,0.9,8.0,7.0


# Features

In [7]:
features: list = ['AQI', 'Status', 'PM10', 'PM10_AVG', 'PM2.5_AVG']
target_feature: str = 'PM2.5'

In [8]:
variables: np.array = np.array([dataset[feature].to_numpy() for feature in features])
variables = variables.transpose()

In [9]:
target: np.array = dataset[target_feature].to_numpy().reshape(1, -1)
target = target.transpose()

In [10]:
train_variables, test_variables, train_target, test_target = train_test_split(variables, target, test_size=0.4, random_state=37710, shuffle=True)

In [11]:
train_variables = shared(train_variables.transpose())
train_target = train_target.transpose()
test_variables = test_variables
test_target = test_target

# Model

In [12]:
with pm.Model() as predictor:
	intercept = pm.Normal(name='intercept', sigma=10)
	coefficients = pm.Normal(name='coefficients', mu=0, sigma=1, shape=5)
	error = pm.HalfCauchy(name='error', beta=5)
	model = pm.Deterministic('model', intercept + pm.math.dot(coefficients, train_variables))
	
	prediction = pm.Normal(name='prediction', mu=model, sigma=error, observed=train_target)

	trace = pm.sample(tune=2000, chains=2, cores=8)
	ppc = pm.sample_posterior_predictive(trace, samples=2000)

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: InplaceDimShuffle{}(TensorConstant{(1,) of 5})
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1918, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1078, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1138, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
      


You can find the C code in this temporary file: /var/folders/vv/93wphtyn4w15b8hssz_f1c500000gn/T/pytensor_compilation_error__z3v9gac
library System is not found.


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: InplaceDimShuffle{}(TensorConstant{(1,) of 5})
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1918, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1078, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1138, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
      


You can find the C code in this temporary file: /var/folders/vv/93wphtyn4w15b8hssz_f1c500000gn/T/pytensor_compilation_error_sv84kfuy
library System is not found.


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: InplaceDimShuffle{}(TensorConstant{(1,) of 5})
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1918, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1078, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1138, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
      

...

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: InplaceDimShuffle{}(TensorConstant{(1,) of 5})
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1918, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1078, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1138, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
      


You can find the C code in this temporary file: /var/folders/vv/93wphtyn4w15b8hssz_f1c500000gn/T/pytensor_compilation_error_daps3d61
library System is not found.


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Elemwise{log,no_inplace}(TensorConstant{5.0})
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1918, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1078, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1138, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
       


You can find the C code in this temporary file: /var/folders/vv/93wphtyn4w15b8hssz_f1c500000gn/T/pytensor_compilation_error_7l4k63b3
library System is not found.

You can find the C code in this temporary file: /var/folders/vv/93wphtyn4w15b8hssz_f1c500000gn/T/pytensor_compilation_error_5z1rhys1
library System is not found.


CompileError: Compilation failed (return status=1):
/usr/local/Caskroom/miniconda/base/envs/BDA/bin/clang++ -dynamiclib -g -Wno-c++11-narrowing -fno-exceptions -fno-unwind-tables -fno-asynchronous-unwind-tables -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -m64 -fPIC -undefined dynamic_lookup -I/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/numpy/core/include -I/usr/local/Caskroom/miniconda/base/envs/BDA/include/python3.11 -I/usr/local/Caskroom/miniconda/base/envs/BDA/lib/python3.11/site-packages/pytensor/link/c/c_code -L/usr/local/Caskroom/miniconda/base/envs/BDA/lib -fvisibility=hidden -o /Users/hermeschen/.pytensor/compiledir_macOS-14.0-x86_64-i386-64bit-i386-3.11.3-64/lazylinker_ext/lazylinker_ext.so /Users/hermeschen/.pytensor/compiledir_macOS-14.0-x86_64-i386-64bit-i386-3.11.3-64/lazylinker_ext/mod.cpp
ld: library not found for -lSystem
clang-15: error: linker command failed with exit code 1 (use -v to see invocation)


# Performance Analysis

In [ ]:
az.plot_trace(trace, var_names=['intercept', 'coefficients', 'error'])

In [ ]:
az.summary(trace, var_names=['intercept', 'coefficients', 'error'])

## MSE

In [78]:
def predict_function(test_variable: np.array) -> float:
	formula_intercept: float = 1
	formula_coefficients: np.array = np.array([1, 1, 1, 1, 1]).reshape(-1, 1)
	return float(np.dot(test_variable.reshape(1, -1), formula_coefficients)) + formula_intercept

In [79]:
predictions: np.array = np.array([predict_function(variable) for variable in test_variables]).reshape(-1, 1)

In [80]:
def mean_square_error(target_prediction: list, target_truth: list) -> float:
	return np.square(np.subtract(target_prediction, target_truth)).mean()

In [82]:
print('MSE: {:.2f}'.format(mean_square_error(predictions, test_target)))

MSE: 4405.61
